In [70]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

from itables import show

In [39]:
agdir = 'CaUsda'

In [96]:
with open(os.path.join(agdir, 'downloads_page.html'), 'r') as f:
    soup = BeautifulSoup(f)

atags = soup.find_all('a', attrs={'aria-label':re.compile('.*agricultural production in CSV')})

links = {}
for atag in atags:
    links[atag['aria-label'][0:4]] = atag['href']

links

{'2020': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2020/main_data_table.csv',
 '2019': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2019/201908cropyear.csv',
 '2018': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2018/2018cactbsErrata.csv',
 '2017': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2017/201708cactb00.csv',
 '2016': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2016/2016cropyear.csv',
 '2015': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2015/2015cropyear.csv',
 '2014': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2014/2014cropyear.csv',
 '2013': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2013/2013cropyear.csv',
 '2012': 'https://www.nass.usda.gov/Statistics_by_State/California/Publications/AgComm/2012/201212cact